In [1]:
from cluster_methods.missingness_cluster import miscluster
from cluster_methods.projection_cluster import projcluster
from model.model import gsrs
from tools.tools import *
from sys import path
import pandas as pd
import numpy as np
import time
import torch
data = pd.read_csv('data/ratings.csv', header=None)

In [62]:
data_sample = data.sample(frac=0.0001)
data_sample.loc[data_sample[0]==550]
# data_sample[0].unique()

array([2544,  481, 2724,  346, 5475,  307, 3106, 2840, 2093,  887, 5389,
       3462,  187, 5578, 4957,  863, 5560, 3761,  490, 3771, 5528, 1114,
       1632, 5282, 1317, 4210, 5942, 5482, 5008, 2931, 4510, 2609, 2026,
       5845, 5794, 1820, 2015, 4448, 4027, 4425, 3310, 4663, 4096, 3829,
        802,  223,  528, 4725, 2986,  195, 1297,  476, 2872, 3997, 4461,
       4562, 2504, 4808,  115, 5693, 5061, 4060, 2843, 5077,  475,  406,
       5925,  663,  409, 1299, 5165, 4959, 3301, 1303,  987, 2446, 5039,
       6014, 4195, 4389, 3695,   25, 3808, 5954, 1626,  333, 5998, 2439,
       1077, 1247, 5831, 4708, 2638, 1958, 2787, 2961, 5575, 4819,  935,
        884])

In [61]:
data_sample.loc[data_sample[0]==2194]

,0,1,2
376173,2194,288,1.0


In [53]:
data_sample_cuda = torch.tensor(data_sample.values).to('cuda')

In [54]:
#divide the data into 10 clusters
ug,ig = miscluster(data_sample, 10)
clusters = {}
clusters['uclusters'] = ug
clusters['iclusters'] = ig

In [55]:
paras = {'lambda':10, 'K':300, 'nstep':50, 'stopall':1e-3, 'stopone':1e-5, 'verbose':False, 'bias': False}
test1 = gsrs(paras)
test1.train(data_sample[[0,1]], data_sample[2], clusters)

In [56]:
result = test1.predict_df(data_sample[[0,1]])

In [57]:
result

,user,item,rating
0,4089,1258,3.817595
1,2453,2769,3.817596
2,2208,2372,2.613506
3,4882,2988,3.817594
4,3778,15,2.025587
...,...,...,...
95,4320,110,4.440844
96,1935,3327,3.817596
97,5026,2359,3.817596
98,5575,367,4.440846


In [67]:
sample_users_ratings_dict = {}
for index,(users,items,ratings) in data_sample.iterrows():
    sample_users_ratings_dict[int(users)] = {}
    sample_users_ratings_dict[users][int(items)] = ratings
sample_users_ratings_dict

{2544: {2353: 4.0},
 481: {3489: 2.0},
 2724: {1487: 2.0},
 346: {1061: 5.0},
 5475: {608: 5.0},
 307: {927: 5.0},
 3106: {1741: 3.0},
 2840: {1562: 2.0},
 2093: {3285: 3.0},
 887: {590: 3.0},
 5389: {2363: 3.0},
 3462: {2915: 3.0},
 187: {1177: 4.0},
 5578: {593: 5.0},
 4957: {553: 3.0},
 863: {2249: 4.0},
 5560: {1573: 5.0},
 3761: {1175: 4.0},
 490: {1030: 4.0},
 3771: {2: 2.0},
 5528: {3250: 5.0},
 1114: {2723: 1.0},
 1632: {2283: 3.0},
 5282: {1197: 4.0},
 1317: {531: 4.0},
 4210: {3623: 4.0},
 5942: {930: 4.0},
 5482: {1207: 4.0},
 5008: {1080: 5.0},
 2931: {858: 5.0},
 4510: {3311: 4.0},
 2609: {1358: 5.0},
 2026: {2136: 2.0},
 5845: {2747: 5.0},
 5794: {1238: 4.0},
 1820: {3671: 4.0},
 2015: {255: 4.0},
 4448: {2762: 5.0},
 4027: {541: 5.0},
 4425: {2417: 2.0},
 3310: {363: 4.0},
 4663: {1079: 4.0},
 4096: {2001: 4.0},
 3829: {474: 4.0},
 802: {1090: 4.0},
 223: {1586: 3.0},
 528: {169: 2.0},
 4725: {592: 3.0},
 2986: {2456: 3.0},
 195: {1844: 4.0},
 1297: {3006: 4.0},
 476: {3

In [70]:
result_users_ratings_dict = {}
for index,(users,items,ratings) in result.iterrows():
    if users not in result_users_ratings_dict:
        result_users_ratings_dict[int(users)] = {}
    result_users_ratings_dict[users][int(items)] = round(ratings,2)
result_users_ratings_dict

{4089: {1258: 3.82},
 2453: {2769: 3.82},
 2208: {2372: 2.61},
 4882: {2988: 3.82},
 3778: {15: 2.03},
 3401: {2719: 2.03},
 3051: {3452: 3.21},
 4250: {140: 3.82},
 3539: {2133: 3.82},
 5403: {2986: 3.82},
 4862: {1408: 3.87},
 3713: {3393: 2.61},
 2753: {2186: 3.82},
 2537: {1214: 3.82},
 3479: {2706: 3.82},
 698: {2657: 3.21},
 2424: {474: 2.61},
 275: {2000: 3.82},
 1632: {1082: 3.21},
 2063: {3316: 2.03, 2046: 3.2},
 5853: {1267: 4.44},
 5005: {3724: 3.21},
 5090: {1408: 4.05},
 3498: {1047: 4.44},
 4849: {919: 4.44},
 3951: {1274: 3.21},
 2258: {590: 3.82},
 3292: {1080: 4.44},
 3411: {3556: 2.61},
 5569: {2571: 3.21},
 4740: {2028: 4.44},
 5595: {288: 3.37},
 1897: {1242: 4.44},
 3609: {52: 3.21},
 5249: {2406: 3.82},
 262: {1129: 3.82},
 3389: {24: 2.61},
 1772: {233: 2.61},
 1697: {1287: 3.82},
 136: {2890: 3.82},
 2748: {2089: 2.03},
 5156: {610: 4.44},
 4425: {1621: 3.21},
 6001: {85: 2.61},
 1088: {1960: 4.44},
 4532: {165: 3.82},
 5523: {2183: 3.82},
 5539: {3341: 3.82},
 